In [38]:
import torch
from transformers import AutoTokenizer, AutoModel
import sqlite3
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\alexx\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

In [40]:
def get_sentence_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=24, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Возьмем усредненное значение всех токенов для представления предложений
    sentence_embeddings = torch.mean(model_output.last_hidden_state, dim=1)
    return sentence_embeddings

In [41]:
# Подключение к базам данных
conn_register = sqlite3.connect('../db/register_of_professions.db')
conn_normalized = sqlite3.connect('../db/normalized_data.db')

# Загрузка данных из баз данных
df_register = pd.read_sql_query("SELECT profession_name FROM proffesions", conn_register)
df_normalized = pd.read_sql_query("SELECT job_name_norm FROM jobs", conn_normalized)

In [42]:
vec_register = get_sentence_embeddings(df_register['profession_name'].tolist())
vec_normalized = get_sentence_embeddings(df_normalized['job_name_norm'].tolist())

In [46]:
vec_register = vec_register.numpy()
vec_normalized = vec_normalized.numpy()

In [49]:
# Словарь для хранения результатов сопоставления
matches = {}

# Вычисление косинусного сходства
for idx, job_vec in enumerate(vec_normalized):
    job_name = df_normalized['job_name_norm'].iloc[idx]
    similarities = cosine_similarity([job_vec], vec_register)[0]
    best_match_index = similarities.argmax()
    best_match = df_register['profession_name'].iloc[best_match_index]
    matches[job_name] = best_match
    print(job_name, ' - ', best_match)

# Результаты сопоставления
matches

ветеринарный врач  -  ветеринарный фельдшер
ветеринарный врач  -  ветеринарный фельдшер
тракторист-машинист  -  машинист экскаватора
тракторист-машинист  -  машинист экскаватора
агроном  -  инженер-мелиоратор
монтажник слаботочных систем  -  монтажник тензорезисторов
механик  -  инженер
электрогазосварщик  -  электрогазосварщик-врезчик
временно исполняющий обязанности главного механика  -  диспетчер электромеханической службы
водитель  -  водитель автомобиля
подсобный рабочий  -  подсобный рабочий
начальник автохозяйства  -  начальник автохозяйства
главный агрохимик  -  главный агрохимик
ветеринарный врач  -  ветеринарный фельдшер
ветеринарный врач  -  ветеринарный фельдшер
дорожный рабочий  -  дорожный рабочий
дорожный рабочий  -  дорожный рабочий
фасовщик продукции  -  распаковщик сырья
дорожный рабочий  -  дорожный рабочий
продавец  -  кассир
сервисный инженер  -  механик по ремонту оборудования
оператор линии стеклопакетов  -  оператор газгольдерной станции
оператор прессов-грануля

KeyboardInterrupt: 